<a href="https://colab.research.google.com/github/DelightInLinght/Project/blob/main/Youtube_Detectection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [17]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation (optional) to improve generalization
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Prepare training and validation data generators
train_generator = train_datagen.flow_from_directory(
    r"/content/Dataset/Train",
    target_size=(299, 299),  # Adjust for your model's input size
    batch_size=64,  # Experiment with batch size
    class_mode='binary'
)
validation_generator = validation_datagen.flow_from_directory(
    r"/content/Dataset/Valid",
    target_size=(299, 299),
    batch_size=64,
    class_mode='binary'
)

Found 173760 images belonging to 2 classes.
Found 28736 images belonging to 2 classes.


In [18]:
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Add, Dropout
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model
from keras.applications import InceptionResNetV2

# Input Layer
input_layer = Input(shape=(299, 299, 3))

# Inception Module
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=input_layer)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Freeze the layers of the InceptionV3 base model
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', restore_best_weights=True)

# Train the model with early stopping
model.fit(
    train_generator,
    epochs=10,  # Adjust based on dataset size and validation performance
    validation_data=validation_generator,
    callbacks=[early_stopping]
)
model.save("Project.h5")


Epoch 1/10
